# Analyze Product Sentiment

In [1]:
import turicreate

# Read product review data

In [2]:
products = turicreate.SFrame('amazon_baby.csv')

Finished parsing file /home/fishyoga/venv/amazon_baby.csv

Parsing completed. Parsed 100 lines in 0.422981 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 105270 lines. Lines per second: 227000

Finished parsing file /home/fishyoga/venv/amazon_baby.csv

Parsing completed. Parsed 183531 lines in 0.610246 secs.

# Explore data

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4


In [4]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [4]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [5]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3


In [6]:
len(giraffe_reviews)

785

In [7]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [8]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [10]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5, '6': 1,'stink': 1, 'because' ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ..."


# Define what is positive and negative sentiment

In [9]:
products['rating'].show()

Materializing SArray

In [10]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [11]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [12]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'recommend': 1.0,'highly': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1


In [13]:
products['sentiment'].show()

Materializing SArray

# Train our sentiment classifier

In [14]:
train_data,test_data = products.random_split(.8,seed=0)

In [15]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 1.879992     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 2.014653     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 2.218663     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 2.951150     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Apply the sentiment classifier to better understand the Giraffe reviews

In [16]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')

In [17]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'recommend': 1.0,'highly': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1


In [18]:
giraffe_reviews = products[products['name']== 'Vulli Sophie the Giraffe Teether']

In [19]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5,"{'purchase': 1.0,'teething': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5,"{'a': 1.0, 'is': 1.0,'when': 1.0, 'him': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1,"{'made': 1.0, 'of': 1.0,'packaging': 1.0, 'no': ...",0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5,"{'another': 1.0, 'out':1.0, 'run': 1.0, 'lost': ...",1
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5,"{'disappointed': 1.0,'will': 1.0, 'take': ...",1
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5,"{'late': 1.0, 'perfect':1.0, 'pack': 1.0, 'on ...",1
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5,"{'delight': 1.0, 'in':1.0, 'other': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5,"{'inside': 1.0, 'water':1.0, 'don': 1.0, 'up': ...",1
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5,"{'its': 1.0, 'fine': 1.0,'is': 1.0, 'which': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,This product is without adoubt the best on the ...,5,"{'belongs': 1.0, 'dog':1.0, 'if': 1.0, 'bewa ...",1


# Sort the Giraffe reviews according to predicted sentiment

In [20]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [21]:
giraffe_reviews

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5,"{'11': 1.0, 'prisrob':1.0, '12': 1.0, 'who': ...",1
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4,"{'around': 1.0,'explore': 1.0, 'they': ...",1
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5,"{'will': 1.0, '15months':1.0, 'would': 2.0, ...",1
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5,"{'out': 1.0, 'would':1.0, 've': 1.0, 'enou ...",1
Vulli Sophie the GiraffeTeether ...,"As every mom knows, youalways want to give your ...",5,"{'whether': 1.0,'neutral': 1.0, 'gend ...",1
Vulli Sophie the GiraffeTeether ...,My Mom-in-Law boughtSophie for my son whe ...,5,"{'penny': 1.0, 'little':1.0, 'perfect': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,"My 4 month old son isteething, and I've tried ...",4,"{'worth': 1.0, 'works':1.0, 'teether': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,Let me just start off byaddressing the choking ...,5,"{'question': 1.0, 'must':1.0, 'overall': 1.0, ...",1
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4,"{'makers': 1.0, 'or':1.0, 'take': 1.0, 'can': ...",1
Vulli Sophie the GiraffeTeether ...,"I admit, I didn't getSophie the Giraffe at ...",4,"{'dye': 1.0, 'of': 1.0,'cause': 1.0, 'fade': ...",1


In [24]:
giraffe_reviews.tail()

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,I bought it for my 8month old (I know it's ...,2.0,"{'and': 5, 'enjoy': 1,'right': 1, 'old': 1, ...",0
Vulli Sophie the GiraffeTeether ...,When we first got it Ithought did my ...,2.0,"{'and': 2, 'cute': 1,'all': 1, 'because': 1, ...",0
Vulli Sophie the GiraffeTeether ...,After getting this for mybaby and after she su ...,1.0,"{'just': 1, 'being': 1,'over': 1, 'chalked': 1, ...",0
Vulli Sophie the GiraffeTeether ...,"My daughter loves Sophie,but we received her as a ...",2.0,"{'and': 2, 'the': 3,""don't"": 1, 'doing': 1, ...",0
Vulli Sophie the GiraffeTeether ...,"""Sophie the Giraffe"" hastested positive for ...",1.0,"{'saying': 1,'nitrosamines.': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I received this in a boxset as a gift. It came ...,2.0,"{'cute': 1, 'unhooked':1, 'being': 2, 'stora ...",0
Vulli Sophie the GiraffeTeether ...,I was intrigued with theidea of Sophie the ...,2.0,"{'baby.)': 1, 'all': 2,'able': 1, 'month': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I wanted to love thisproduct and was excited ...,1.0,"{'just': 1, 'may,': 1,'children': 1, 'bummed': ...",0
Vulli Sophie the GiraffeTeether ...,This children's toy isnostalgic and very cute. ...,1.0,"{'expectations.': 1,'refund': 1, 'there': 1, ...",0
Vulli Sophie the GiraffeTeether ...,"My son (now 2.5) LOVEDhis Sophie, and I bought ...",1.0,"{'all': 1, 'just': 1,""don't"": 1, 'over': 2, ...",0


## Show the most positive reviews

In [22]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [23]:
giraffe_reviews[1]['review']

'I\'ll be honest...I bought this toy because all the hip parents seem to have one too and I wanted to be a part of the "hip parent" crowd. The price-tag was somewhat of a deterent but I prevailed and purchased this teether for my daughter.At first, Lily didn\'t know what to make of of Sophie and showed little interest in the polka-dotted creature. I continued to introduce Lily to Sophie and kept the toy in the carrier so that it was on-hand during transitions. Eventually, Lily discovered what a wonderful experience it was to gnaw on the hooves and ears and these two have never been far apart since.Lily really enjoys gumming all the different parts of Sophie like no other teether we have. The size of the toy is great as it is somewhat substantial and so easy for a little one to grasp and hold onto. Lily really enjoys hearing Sophie squeak and will smile whenever Sophie makes a noise or pops her head up from Mommy\'s lap to say hello.People have stopped and commented on Sophie and to the

# Most negative reivews

In [24]:
giraffe_reviews[-1]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [25]:
giraffe_reviews[-2]['review']

'I wanted to love this product and was excited to buy it when I became pregnant but am now hesitant to let my baby use it after reading about the recall in Europe. Apparently, as I understand it, their toxin standards of measurement are lower than ours so they have not been recalled here (apparently we are OK with low levels of nitrates in the toys our children put in their mouths, but Europeans are not...hmmm)...Be that as it may, toxins registering even CLOSE to a dangerous level made me nervous about using. After digging around online I did discover that the company claims to have changed the product after a certain date and lists manufacturing codes so you can check yours (those listed were made after a certain date and are said to be safer). Sadly mine was not made after the &#34;improved&#34; date but I could not return it because there was no formal recall in our country. I considered returning it and hunting for one with an approved manufacturing date but man that was just too 

# Assignments

In [28]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [29]:
def mjt_count(word, word_counts):
    if word in word_counts:
        return word_counts[word]
    else:
        return 0

In [32]:
for word in selected_words:
    products[word]= products['word_count'].apply(lambda word_dict: mjt_count(word, word_dict))

In [36]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'recommend': 1.0,'highly': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1


## word most and least used

In [38]:
for word in selected_words:
    print (word, products[word].sum())

awesome 3892.0
great 55791.0
fantastic 1664.0
amazing 2628.0
love 41994.0
horrible 1110
bad 4183
terrible 1146.0
awful 687
wow 425
hate 1107


In [41]:
train_data,test_data = products.random_split(.8, seed=0)

In [43]:
mjt_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=selected_words, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.111078     | 0.847708          | 0.846385            |

| 5         | 6        | 0.135313     | 0.847708          | 0.846385            |

| 6         | 7        | 0.160515     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

## most postive and negative weight

In [49]:
coef = mjt_model.coefficients

In [62]:
coef.sort('value', ascending=False)

name,index,class,value,stderr
love,None,1,1.3592688669225095,0.028068300152099074
(intercept),None,1,1.336591384887764,0.008929969787656441
awesome,None,1,1.1335346660341385,0.0839964398318753
amazing,None,1,1.1000933113660234,0.09954776260465989
fantastic,None,1,0.8858047568814243,0.1116759129339967
great,None,1,0.8630655001196581,0.018955052444376858
wow,None,1,-0.009538236067684405,0.1604641122471165
bad,None,1,-0.9914778800650621,0.038484286646990606
hate,None,1,-1.3484407222463166,0.07715698604297333
awful,None,1,-2.0529082040313567,0.10099735435259254


In [82]:
coef.sort('value', ascending=True)

name,index,class,value,stderr
horrible,None,1,-2.251335236759098,0.08020249388788436
terrible,None,1,-2.223661436085132,0.07731736203785752
awful,None,1,-2.0529082040313567,0.10099735435259254
hate,None,1,-1.3484407222463166,0.07715698604297333
bad,None,1,-0.9914778800650621,0.038484286646990606
wow,None,1,-0.009538236067684405,0.1604641122471165
great,None,1,0.8630655001196581,0.018955052444376858
fantastic,None,1,0.8858047568814243,0.1116759129339967
amazing,None,1,1.1000933113660234,0.09954776260465989
awesome,None,1,1.1335346660341385,0.0839964398318753


## accuracy

In [53]:
results=mjt_model.evaluate(test_data)

In [54]:
results

{'accuracy': 0.8463848186404036,
 'auc': 0.6935096220934976,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.39622654670874075,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 5328

## majority class classifier accuracy

In [86]:
27817/(27817+4957+371+159)

0.8352450156137401

In [56]:
results2=sentiment_model.evaluate(test_data)

In [57]:
results2

{'accuracy': 0.9176975738650012,
 'auc': 0.9258242975424673,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.3304787187240534,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0  

In [65]:
mjt_model.evaluate(test_data)  # no need to set results

{'accuracy': 0.8463848186404036,
 'auc': 0.6935096220934976,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.39622654670874075,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 5328

In [66]:
mjt_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 5328 |
 |   0.001   | 0.9994369369369369 | 1.0 | 27976 | 5328 |
 |   0.002   | 0.9992492492492493 | 1.0 | 27976 | 5328 |
 |   0.003   | 0.9990615615615616 | 1.0 | 27976 | 5328 |
 |   0.004   | 0.9990615615615616 | 1.0 | 27976 | 5328 |
 |   0.005   | 0.9981231231231231 | 1.0 | 27976 | 5328 |
 |   0.006   | 0.9981231231231231 | 1.0 | 27976 | 5328 |
 |   0.007   | 0.9979354354354354 | 1.0 | 27976 | 5328 |
 |   0.008   | 0.9973723723723724 | 1.0 | 27976 | 5328 |
 |   0.009   | 0.9971846846846847 | 1.0 | 27976 | 5328 |
 +-----------+--------------------+-----+-------+------+
 [1001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(nu

In [67]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0    |        1.0         |        1.0         | 27976 | 5328 |
 |   0.001   | 0.698948948948949  | 0.9933514440949385 | 27976 | 5328 |
 |   0.002   | 0.6619744744744744 | 0.9922076065198742 | 27976 | 5328 |
 |   0.003   | 0.6385135135135135 | 0.9909922790963683 | 27976 | 5328 |
 |   0.004   |  0.62256006006006  | 0.9903846153846154 | 27976 | 5328 |
 |   0.005   | 0.6112987987987988 | 0.989848441521304  | 27976 | 5328 |
 |   0.006   | 0.6002252252252253 | 0.9894195024306548 | 27976 | 5328 |
 |   0.007   | 0.589527027027027  | 0.9888475836431226 | 27976 | 5328 |
 |   0.008   | 0.5816441441441441 | 0.9883828996282528 | 27976 | 5328 |
 |   0.009   | 0.574512012012012

In [68]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'recommend': 1.0,'highly': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1


In [69]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [70]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4,"{'convenient': 1.0,'more': 1.0, 'trash': ...",1
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1,"{'system': 1.0, 'try':1.0, 're': 1.0, 'still': ...",0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5,"{'nose': 1.0, 'for': 2.0,'investment': 1.0, ...",1
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5,"{'out': 1.0, 'pull': 1.0,'open': 1.0, 'pail': ...",1
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5,"{'winter': 1.0,'outside': 1.0, 'day': ...",1
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4,"{'mom': 1.0, 'my': 1.0,'empty': 2.0, 'poop': ...",1
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1,"{'yuck': 1.0, 'clean':1.0, 'all': 1.0, 'tra ...",0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5,"{'price': 1.0,'suggestions': 1.0, ...",1
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5,"{'best': 1.0, 'that':1.0, 'handle': 1.0, ' ...",1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4,"{'no': 1.0, 'regular':1.0, 'part': 1.0, ...",1


In [71]:
len(diaper_champ_reviews)

298

In [72]:
count_row = diaper_champ_reviews.shape[0]

In [73]:
count_row

298

In [74]:
diaper_champ_reviews.shape[1]

17

In [75]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type = 'probability')

### Consider the most positive review for ‘Baby Trend Diaper Champ’ according to the sentiment_model from the IPython Notebook from lecture. Which of the following ranges contains the predicted_sentiment for this review, if we use the selected_words_model to analyze it?

In [84]:
diaper_champ_reviews=diaper_champ_reviews.sort('predicted_sentiment')

In [85]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1,"{'system': 1.0, 'try':1.0, 're': 1.0, 'still': ...",0
Baby Trend Diaper Champ,I LOVED my Champ for thefirst year as it was all ...,2,"{'time': 1.0, 'due': 1.0,'duty': 1.0, 'fails': ...",0
Baby Trend Diaper Champ,Worst diaper pale ever!!I've had mine for 2 y ...,1,"{'this': 1.0,'unexpectedly': 1.0, ...",0
Baby Trend Diaper Champ,This thing freakin'stinks. Literally. I ...,1,"{'them': 1.0, 'to': 1.0,'listened': 1.0, 'have': ...",0
Baby Trend Diaper Champ,This is the worst diaperpail ever! It was great ...,1,"{'not': 1.0, 'does': 1.0,'well': 1.0, 'hopeful': ...",0
Baby Trend Diaper Champ,"Well, the first threemonths were odor free!! ...",2,"{'have': 1.0, 'don': 1.0,'they': 1.0, ...",0
Baby Trend Diaper Champ,My 8 year old yellow labwas able to get the top ...,1,"{'prevent': 1.0,'literature': 1.0, ...",0
Baby Trend Diaper Champ,I am really disappointedwith the Diaper Champ. ...,2,"{'pails': 1.0, 'look':1.0, 'will': 1.0, 'ba ...",0
Baby Trend Diaper Champ,This product is nice intheory but not all that ...,1,"{'a': 1.0, 'else': 1.0,'buy': 1.0, 'save': 1.0, ...",0
Baby Trend Diaper Champ,"Like other reviewers, Iput this diaper pail on ...",5,"{'plastic': 1.0,'recycled': 1.0, ...",1


In [78]:
diaper_champ_reviews['mjt_predicted_sentiment'] = mjt_model.predict(diaper_champ_reviews, output_type = 'probability')

In [83]:
diaper_champ_reviews.sort('mjt_predicted_sentiment', ascending=True)

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,......all I can say isthe smell is ...,1,"{'buy': 1.0, 'this': 1.0,'t': 1.0, 'don': 1.0, ...",0
Baby Trend Diaper Champ,My 8 year old yellow labwas able to get the top ...,1,"{'prevent': 1.0,'literature': 1.0, ...",0
Baby Trend Diaper Champ,I registered for thisproduct after reading ...,2,"{'lift': 1.0, 'nails':1.0, 'three': 1.0, ...",0
Baby Trend Diaper Champ,For my first born Ipurchased the Diaper ...,5,"{'house': 1.0, 'still':1.0, 'is': 1.0, 'this': ...",1
Baby Trend Diaper Champ,The Diaper Champ isTERRIBLE at keeping the ...,1,"{'just': 1.0, 'dirty':1.0, 'enjoy': 1.0, ...",0
Baby Trend Diaper Champ,I've read all of thereviews of those of you ...,2,"{'never': 1.0, 'day':1.0, 'buy': 1.0, ...",0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5,"{'winter': 1.0,'outside': 1.0, 'day': ...",1
Baby Trend Diaper Champ,It's so much better thenthe diaper genie. No ...,5,"{'or': 1.0, 'fixed': 1.0,'be': 1.0, 'can': 1.0, ...",1
Baby Trend Diaper Champ,I nannied for two babiesin some one else's home ...,2,"{'odor': 1.0, 'with':1.0, 'has': 1.0, 'the ...",0
Baby Trend Diaper Champ,This is one of the bestthings that we got as a ...,5,"{'live': 1.0, 'can': 1.0,'still': 1.0, 'bag': ...",1
